In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import cPickle as pk
import pandas as pd
import numpy as np

# PATSTAT Data Checks

# Table of Contents
* [Preliminary Data Munge](#Prelim-Data-Munge)
* [Importing Data](#Importing)
* [Distribution of Patents in Time by Publication Year](#Distribution-of-Patents-by-Pub-Yr)
* [By Publication Year (Split-apply-combine)](#Split-apply-combine-On-Publication-Year)
    * [Distribution Of Patents In Time by Pub Year](#Distribution-in-Time-by-Pub-Check/Practice)
    * [Checking Application Dates](#Checking-Application-Dates)
    * [Checking Classfications](#Checking-Classfications)
* [By Authory-Publication Year (Split-apply-combine)](#Split-apply-combine-On-Authority-And-Publication-Year)
    * [Distribution of Patents In Time by Authority-Pub Year](#Quick–number-of-patents-authority-pub-year)
    * [Checking Application Dates](#Checking-Application-Dates-By-Auth-Year)
    * [Checking IPC Classfications](#Checking-IPC-Classfications-By-Auth)
    * [Checking CPC Classfications](#Checking-CPC-Classfications-By-Auth)

## Prelim Data Munge
[to top](#Table-of-Contents) 

In [ ]:
# Take date col. with format 'yyyy-mm-dd' and return that col as int(yyyy)
def d_to_y(col):
    return col.map(lambda x:int(x[:4]))

In [ ]:
#Date to years function - works on application date and publication date in one
def date_to_year(df):
    if 'appln_filing_date' in df.columns:
        df.appln_filing_date = df.appln_filing_date.map(lambda x:int(x[:4]))
        df.rename(columns={'appln_filing_date':'app_year'}, inplace=True)
    if 'publn_date'in df.columns:
        df.publn_date = df.publn_date.map(lambda x:int(x[:4]))
        df.rename(columns={'publn_date':'pub_year'}, inplace=True)
    return

In [ ]:
# Date to binary is missing or now
def date_to_binary(df, d_col, new_col_name):
    if d_col in df.columns:
        df[d_col] = df[d_col].map( lambda x: 0 if x == 9999 else 1 )
        df.rename(columns={d_col:new_col_name}, inplace=True)
    else: 
        return 'd_col not a column'

## SQL Query used on PATSTAT 2015 Spring

## Importing
[to top](#Table-of-Contents) 

In [ ]:
pwd

In [ ]:
f_all_grants = '/Users/innovation-sfi/Dropbox/Research/Innovation/EPO/Data_processing/Data_Play/all_grant_only_prior_flags_v1.csv'

In [ ]:
df = pd.read_csv(f_all_grants, engine='c', usecols=[1,2,3,5,6]); df.head()
#dont need ID's

In [ ]:
# munge dates to years
date_to_year(df); df.head()

In [ ]:
# munge application to year to binary if date is missing (app_year == 9999)
date_to_binary(df, 'app_year', 'has_app_date'); df.head()

In [ ]:
df[df['appln_auth'] == 'EP'].sort_values(by='pub_year', ascending=True).head()

In [ ]:
df_pre_ep = df[df['pub_year'] < 1980]; df_pre_ep.head()

In [ ]:
# Find top 6 countries by number of patents, and get list of those autheorties for later
auth_grp = df.groupby('appln_auth')
auth_num = len(auth_grp)
auth_sizes = auth_grp.size()
auth_sizes.sort_values(ascending=False, inplace=True)
print auth_sizes.head()
auth_by_size = auth_sizes.index.values
print auth_by_size

In [ ]:
# PRE EP Find top 6 countries by number of patents, and get list of those autheorties for later
auth_grp_pre_ep = df_pre_ep.groupby('appln_auth')
auth_num_pre_ep = len(auth_grp_pre_ep)
auth_sizes_pre_ep = auth_grp_pre_ep.size()
auth_sizes_pre_ep.sort_values(ascending=False, inplace=True)
print auth_sizes_pre_ep.head()
auth_by_size_pre_ep = auth_sizes_pre_ep.index.values
print auth_by_size_pre_ep

In [ ]:
list_1 = auth_sizes[0:31]
N = len(list_1)
fs = np.array(range(N))/float(N)
xs = list_1.sort_values(ascending=False).values
print xs[0]
plt.plot(xs, fs)
ax = plt.gca()
#ax.set_yscale('log')
ax.set_xscale('log')
#plt.savefig('figures/all_grant_auth_dists_v1.pdf')

In [ ]:
N = len(auth_sizes)
fs = np.array(range(N))
xs = auth_sizes.sort_values(ascending=False).values
print xs[0]
plt.plot(fs, xs)
ax = plt.gca()
#ax.set_yscale('log')
ax.set_xscale('log')
#plt.savefig('figures/all_grant_auth_dists_v1.pdf')

In [ ]:
N = len(auth_sizes_pre_ep)
fs = np.array(range(N))/float(N)
xs = auth_sizes_pre_ep.sort_values(ascending=False).values
plt.plot(xs, fs)
ax = plt.gca()
ax.set_yscale('log')
ax.set_xscale('log')
#plt.savefig('figures/all_grant_auth_dists_pre_ep_v1.pdf')

In [ ]:
# Get European authority codes
with open('/Users/innovation-sfi/Desktop/eu_auths.txt') as f:
    eu_auths = f.read().splitlines()
# clean off those not contained in patstat
eu_auths = [val for val in auth_by_size if val in eu_auths]
# add 'EP' to the list
eu_auths.insert(0,'EP')
print eu_auths

In [ ]:
split = int((auth_sizes.EP/float(len(eu_auths)-1))); print split
auth_sizes_split_ep = auth_sizes; #print auth_sizes_split_ep
auth_sizes_split_ep = auth_sizes_split_ep[auth_sizes_split_ep.index != 'EP']; #print auth_sizes_split_ep
print auth_sizes_split_ep.loc['FR']
print auth_sizes_split_ep.loc['FR'] + split
for eu in eu_auths:
    if eu != 'EP':
        auth_sizes_split_ep.loc[eu] = auth_sizes_split_ep.loc[eu] + split
print auth_sizes_split_ep.head()

In [ ]:
N = len(auth_sizes_split_ep)
fs = np.array(range(N))/float(N)
xs = auth_sizes_split_ep.sort_values(ascending=False).values
plt.plot(xs, fs)
ax = plt.gca()
ax.set_yscale('log')
ax.set_xscale('log')
#plt.savefig('figures/all_grant_auth_dists_eq_split_ep_v1.pdf')